Uvoz knjižnic in branje podatkov iz datotek:

In [ ]:
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np
import operator

In [ ]:
list_of_cvs = ["./podatki/kd{}.csv".format(i) for i in range(2010,2020)]
frames = [pd.read_csv(csv,sep=';', encoding="UTF-8", dtype={'MesecStoritve': str}) for csv in list_of_cvs]
df = pd.concat(frames)

In [ ]:
dfGroupedBySex = df.groupby(["Spol"]);

In [ ]:
# { sex: {year: count} }
crimesBySex = {};
for sex, bySexData in dfGroupedBySex:
    if(sex not in crimesBySex):
        crimesBySex[sex] = defaultdict(int);
    for yearStr in bySexData["MesecStoritve"].values:
        year = int(yearStr.split(".")[1]);
        crimesBySex[sex][year] += 1;
#print(crimesBySex);

In [ ]:
plt.figure(figsize=(15,10));
for sex, crimesPerYear in crimesBySex.items():
    if(sex != "MOŠKI" and sex != "ŽENSKI"):
        continue;
    sortedYears = [ year for year in sorted(crimesPerYear.keys()) if(2010 <= year < 2020) ];
    crimesPerSortedYear = [ crimesPerYear[year] for year in sortedYears ];
    plt.plot(sortedYears, crimesPerSortedYear, label=sex, linewidth=2.0);
plt.legend();

Kazniva dejanja po letih:

In [ ]:
#group by mesec, count unique id
kd_leto = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD'])
leto_counter = defaultdict(int)

for name,v in kd_leto:
    index = int((name[0].split(".")[1]))
    if index >= 2010 and index < 2020:
        leto_counter[index] +=1
    
    
list_mes_counter = sorted(leto_counter.items())
mesec, counter = zip(*list_mes_counter)
plt.figure(figsize=(15,10))
plt.plot(mesec,counter)
plt.show()

Porazdelitev kaznivih dejanj po mesecih:

In [ ]:
kd_mes = df.groupby(['MesecStoritve', 'UraStoritve','ZaporednaStevilkaKD'])
mes_counter = defaultdict(int)

for name,v in kd_mes:
    index = int((name[0].split(".")[0]))
    mes_counter[index] +=1


list_mes_counter = sorted(mes_counter.items())
mesec, counter = zip(*list_mes_counter)
plt.figure(figsize=(15,10))
plt.bar(mesec,counter)
plt.show()

Udeleženci v kaznivih dejanjih porazdeljeni po starostnih razredih:

In [ ]:
star_razredi = df.groupby(['StarostniRazred'])
star_razred, star_stevilo = zip(*[(name,razred.size) for name, razred in star_razredi])

star_stevilo = list(star_stevilo)
star_stevilo[-3] += star_stevilo[-2]

plt.figure(figsize=(15,10))
plt.bar(star_razred[:-2],star_stevilo[:-2])
plt.show()
#plt.pie(star_stevilo[:-3], labels=star_razred[:-3])

In [ ]:
#group by mesec, count unique id
kd_ura = df.groupby(['UraStoritve', 'ZaporednaStevilkaKD'])
ura_counter = defaultdict(int)

for name,v in kd_ura:
    ura_counter[name[0]] += 1


list_ura_counter = sorted(ura_counter.items())
ura, counter = zip(*list_ura_counter)
plt.figure(figsize=(40,10))
plt.bar(ura,counter, color="black")
plt.show()

In [ ]:
from PIL import Image
#pip install reportlab
from reportlab.graphics import renderPM
#pip install svglib
from svglib.svglib import svg2rlg

def getPILImage( pathToSVG ):
    drawing = svg2rlg( pathToSVG );
    return renderPM.drawToPILP( drawing );


import re

def getColorFronNormVal( normVal ):
    p = 1.0 - normVal;
    r = min( 255, p * 2 * 255 );
    g = min( 255, normVal * 2 * 255 );
    return "#%02x%02x%02x" % ( int(r), int(g), 0 );

def puMapingService( puName ):
    if ( puName == "PU LJUBLJANA" ):
        return "PU-LJ";
    elif  ( puName == "PU CELJE" ):
        return "PU-CE";
    elif  ( puName == "PU KRANJ" ):
        return "PU-KR";
    elif  ( puName == "PU KOPER" ):
        return "PU-KP";
    elif  ( puName == "PU KRANJ" ):
        return "PU-KR";
    elif  ( puName == "PU MARIBOR" ):
        return "PU-MB";
    elif  ( puName == "PU MURSKA SOBOTA" ):
        return "PU-MS";
    elif  ( puName == "PU NOVA GORICA" ):
        return "PU-NG";
    elif  ( puName == "PU NOVO MESTO" ):
        return "PU-NM";
    else:
        return puName;

def getModifiedPUStats( stats ):
    return { puMapingService(pu): getColorFronNormVal(normVal) for pu,normVal in stats.items() }
    
def transformSvgPathLine( line, fillColor ):
    return line.rstrip()[:-1] + " fill=\"%s\">\n" % fillColor;
    
# stats = { "PU_X": normVal, ...  }
def getModifiedSVG( pathToSvg, stats ):
    colorPerPU = getModifiedPUStats( stats );
    svg = "";
    with open(pathToSvg) as file:
        for line in file:
            if ( line.lstrip().startswith( "<path d=" ) ):
                matchedID = re.findall( 'id="PU-..', line );
                if ( len( matchedID ) != 1 ):
                    print( "ERROR in ", line, matchedID );
                    svg += line;
                    continue;
                pu = re.findall( "PU-..", matchedID[0] )[0];
                if ( pu not in colorPerPU ):
                    #print("Unknown PU:", pu, "- will have default color fill");
                    svg += line;
                    continue;
                svg += transformSvgPathLine( line, colorPerPU[pu] );
            else:
                svg += line;
    return svg;

def getFrame( stats ):
    pathToSIPUFile = "./misc/si_pu.svg";
    pathToTempSvg = "./misc/temp.svg";
    svg = getModifiedSVG( pathToSIPUFile, stats );
    with open( pathToTempSvg, "w" ) as svgFile:
        svgFile.write( svg );
    return getPILImage( pathToTempSvg );

# allStats = [ (datum, {'PU': normVal, ...}), ... ]
def getGif( allStats ):
    frames = [];
    for datum, stats in allStats:
        frames.append( getFrame( stats ) );
    return frames;
    
def saveGif( frames, path ):
    frames[0].save(path, save_all=True, append_images=frames[1:], optimize=False, duration=150, loop=1)

def generateGif( allStats, gifPath ):
    saveGif( getGif( allStats ), gifPath );

    
    

tempAllStats =  [ ('0.0', { "PU LJUBLJANA": 0.2 }), ('0.1', { "PU CELJE": 0.2 }) ];
generateGif( tempAllStats, "./misc/temp.gif" );


In [ ]:
pu = df.groupby(['PUStoritveKD'])

for name, v in pu:
    print(name, len(v))

#print(pu.ngroups)


In [ ]:
df["TIP-DEJANJA"] = df.apply(lambda row: row["OpisKD"].split(" - ")[1], axis=1)

In [ ]:
kd = df.groupby(["TIP-DEJANJA"])
kd.ngroups

tip_dogodki = defaultdict(set)

for k, v in kd:
    if k== "": continue
    for val in v.values:
        if val[4] in ["GPU", "DIREKTORAT ZA UPRAVNE NOTRANJE ZADEVE", "NI PODATKA"]: continue
        tip_dogodki[k].add((val[4],val[1] ,val[0], val[2], int(val[1].split(".")[1]), int(val[1].split(".")[0])))



tip_urejeni_dogodki = {k:(sorted([zapis for zapis in v], key=lambda x: (x[4],x[5]))) for k,v in tip_dogodki.items()}


In [ ]:
for tema,v in tip_urejeni_dogodki.items():
    if len(v) <5:
        print(tema)
        for v1 in v:
            print("     "+ str(v1))
        break

In [ ]:
#[(datum,{PU:vrednost,...})]
enote = ["PU CELJE","PU KOPER","PU KRANJ","PU LJUBLJANA","PU MARIBOR","PU MURSKA SOBOTA","PU NOVA GORICA","PU NOVO MESTO"]
vse = []
for dejanje, vrednosti in tip_urejeni_dogodki.items():

    meseci = []
    sdatum = None
    smesec = None
    sleto = None
    enote_norm = {enota:0 for enota in enote}

    #if len(vrednosti) > 1000 or len(vrednosti) < 100: continue

    for vrednost in vrednosti:
        if vrednost[4] < 2010:
            continue
        if sdatum == None:
            sdatum=vrednost[1]
            sleto = vrednost[4]
            smesec = vrednost[5]
        if vrednost[1] != sdatum:
            while ((vrednost[4]*12 + vrednost[5]) - (sleto*12 + smesec)) > 1:
                smesec +=1
                if smesec > 12:
                    smesec = 1
                    sleto +=1
                if smesec < 10:
                    meseci.append(("0{}.{}".format(smesec,sleto), {enota:0 for enota in enote}))
                else:
                    meseci.append(("{}.{}".format(smesec,sleto), {enota:0 for enota in enote}))

            maxi = np.max(np.array([_v for _k,_v in enote_norm.items()]))
            enote_norm_tmp = {e:round(v/maxi,4) for e,v in enote_norm.items()}
            meseci.append((vrednost[1], enote_norm_tmp))
            #enote_norm = {enota:0 for enota in enote}
            sdatum=vrednost[1]
            sleto = vrednost[4]
            smesec = vrednost[5]

        enote_norm[vrednost[0]] +=1
    #print(meseci)
    vse.append((dejanje,meseci))

for dejanje,meseci in vse:
    if dejanje in ['TATVINA','VELIKA TATVINA','GOLJUFIJA','KRŠITEV TEMELJNIH PRAVIC DELAVCEV','POSLOVNA GOLJUFIJA','POŠKODOVANJE TUJE STVARI','NEPLAÈEVANJE PREŽIVNINE','NASILJE V DRUŽINI','GROŽNJA','LAHKA TELESNA POŠKODBA']:
        generateGif( meseci, "./misc/{}.gif".format(dejanje) );